## import of swagger client / calling auth function on Server to get bearer token 

In [ ]:

import swagger_client
import json
from swagger_client.rest import ApiException

configuration = swagger_client.Configuration()



api_client = swagger_client.ApiClient(configuration=configuration)

# create an instance of the API class
api_instance = swagger_client.AuthenticationApi()
mylogin = swagger_client.LoginDto(email = 'java@sse.de', password = 'pw')
try:
    # Get list of exposure types
    print(api_instance.api_client.configuration.api_key_prefix)
    api_response = api_instance.auth_controller_login(body = mylogin, async_req=True)
    token = api_response.get().access_token 
    api_client.set_default_header('Authorization', 'Bearer ' + api_response.get().access_token)
except ApiException as e:
    print("Exception when calling Api-> %s\n" % e)




## setting bearer token / list repositories 

In [ ]:
repApi = swagger_client.CompetenciesApi(api_client)

print (token)

try:
    # Get list of exposure types
    api_response = repApi.repository_mgmt_controller_list_repositories()
    print(api_response)
    print(repApi.api_client.configuration.host)
except ApiException as e:
    print("Exception when calling Api-> %s\n" % e)
# create an instance of the API class


## Search for Rep

In [ ]:
repDto = swagger_client.RepositorySearchDto(page_size= 1.0,page= 1.0,name= '1')

try:
    # Get list of exposure types
    api_response = repApi.repository_mgmt_controller_search_for_repositories(repDto)
    print(api_response)
except ApiException as e:
    print("Exception when calling Api-> %s\n" % e)
# create an instance of the API class



## Load Competences by RepoId

In [ ]:



try:
    # Get list of exposure types
    api_response = repApi.repository_mgmt_controller_load_repository(repository_id=1)
    print(api_response)
except ApiException as e:
    print("Exception when calling Api-> %s\n" % e)
# create an instance of the API class

## Load resolved competences

In [ ]:
try:
    # Get list of exposure types
    my_competencies = repApi.repository_mgmt_controller_load_resolved_repository (repository_id=1)
    print(my_competencies)
except ApiException as e:
    print("Exception when calling Api-> %s\n" % e)
# create an instance of the API class

## Goals for Repository 

In [ ]:
loApi = swagger_client.LearningObjectsApi()


try:
    my_goals = loApi.lo_repository_controller_show_goal(goal_id=1)
    print(my_goals)
except ApiException as e:
    print("Exception when calling Api-> %s\n" % e)

## goal by ID

In [ ]:
try:
    api_response = loApi.lo_repository_controller_show_goal(goal_id=1)
    print(api_response)
except ApiException as e:
    print("Exception when calling Api-> %s\n" % e)

## load LO Repo by Id

In [ ]:
try:
    my_lo = loApi.lo_repository_controller_load_repository(repository_id=1)
    print(my_lo)
except ApiException as e:
    print("Exception when calling Api-> %s\n" % e)

## load LO by Id

In [ ]:
my_LO_complete = []   
for lo in my_lo.learning_objects:    
    try:
        print(lo.id)
        my_LO_complete.append(loApi.lo_repository_controller_load_learning_object (learning_object_id=lo.id))

    except ApiException as e:
        print("Exception when calling Api-> %s\n" % e)
print(my_LO_complete)

In [ ]:
def find_comp_by_id(in_id):
    for comp in my_competencies.competencies:
        if (comp.id==in_id):
            print(comp)
            break
def find_something_by_id(arr , id):
            return next(x for x in arr if x.id == id )


In [ ]:

for comp in my_goals.competencies:
    find_something_by_id(my_competencies.competencies, comp)

for comp in my_goals.uber_competencies:
    find_something_by_id(my_competencies.ueber_competencies, comp)  

In [ ]:
def find_LO_for_competence(competence):
    for lo in my_lo.learning_objects:
        try:
            api_response = loApi.lo_repository_controller_load_learning_object(learning_object_id=lo.id)
            print(api_response.name)
        except ApiException as e:
            print("Exception when calling Api-> %s\n" % e)

In [ ]:
find_LO_for_competence('1')

## Graphen aus Daten erzeugen

In [ ]:
import networkx as nx
gr = nx.DiGraph()

In [ ]:
def addEdgeforList(comp, graph ):
    for nestedcomp in comp.nested_competencies:
        graph.add_edge(nestedcomp.id,comp.id+'ueber', art='neestedComp' )
    for nestedubercomp in comp.nested_ueber_competencies:    
        graph.add_edge(nestedubercomp.id+'ueber',comp.id+'ueber', art='neestedueberComp')
        print('add ',nestedubercomp.id+'ueber',comp.id+'ueber')
        if nestedubercomp.nested_ueber_competencies:
            for nestcomp in nestedubercomp.nested_ueber_competencies:
                addEdgeforList(nestcomp, graph )

In [ ]:
for comp in my_competencies.competencies:
    gr.add_node(comp.id, name=comp.description, art='Kompetenz')
for comp in my_competencies.ueber_competencies:
    gr.add_node(comp.id+'ueber', name=comp.description, art='Ueber_Kompetenz')
    addEdgeforList(comp, gr )

In [ ]:
gr.nodes.data()

In [ ]:
print('Comp:')
print(my_competencies.competencies)
print('ÜberComp:')
print(my_competencies.ueber_competencies)
print('LO:')
print(my_lo)
print('LO Comp:')
print (my_LO_complete)

In [ ]:
gr.edges

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,20)

In [ ]:
subax1 = plt.subplot(111)
nx.draw(gr, with_labels=True, font_weight='bold')

In [ ]:
grLO = nx.DiGraph()
grLO.add_node('Know nothing', subset='Start')
for comp in my_competencies.competencies:
    grLO.add_node(comp.id, name=comp.description, subset='Kompetenz')
for comp in my_competencies.ueber_competencies:
    grLO.add_node(comp.id+'ueber', name=comp.description, subset='Ueber_Kompetenz')
    addEdgeforList(comp, grLO )
for lo in my_LO_complete:
    grLO.add_node(lo.id+'Lo', name=lo.description, subset='LO')
    if not lo.required_competencies:
        if not lo.required_ueber_competencies: 
            grLO.add_edge( 'Know nothing',lo.id+'Lo', art = 'required_comp')
    for nestcomp in lo.offered_competencies:
        grLO.add_edge(lo.id+'Lo', nestcomp, art = 'offered_comp')
    for nestcomp in lo.required_competencies:
        grLO.add_edge(nestcomp,lo.id+'Lo', art = 'required_comp')    
    for nestcomp in lo.offered_ueber_competencies:
        grLO.add_edge(lo.id+'Lo', nestcomp+'ueber', art = 'offered_ueber_comp')
    for nestcomp in lo.required_ueber_competencies:
        grLO.add_edge(nestcomp+'ueber',lo.id+'Lo', art = 'required_ueber_comp')  

In [ ]:
subax1 = plt.subplot(111)
nx.draw(grLO, with_labels=True, font_weight='bold')

In [ ]:
subax1 = plt.subplot(111)
pos = nx.circular_layout(grLO)

edges = grLO.edges()
colors = [grLO[u][v]['art'] for u,v in edges]
for i in range(len(colors)):
  
    if colors[i] == 'neestedComp':
        colors[i] = 'r'
  
    if colors[i] == 'required_comp':
        colors[i] = 'g'
  
    if colors[i] == 'required_ueber_comp':
        colors[i] = 'b'
  
    if colors[i] == 'neestedueberComp':
        colors[i] = 'c'
  
    if colors[i] == 'offered_comp':
        colors[i] = 'm'
  
    if colors[i] == 'offered_ueber_comp':
        colors[i] = 'y'
  
# print list
print(colors)
pos = nx.circular_layout(grLO)

nx.draw(grLO, pos=pos,edge_color=colors, with_labels=True)

In [ ]:
grLO_reversed = nx.reverse(grLO)
nx.draw(grLO_reversed,edge_color=colors, with_labels=True)

In [ ]:
grLO_reversed.edges

In [ ]:
paths = nx.all_simple_paths(grLO, source='Know nothing', target='9ueber', cutoff=50)

In [ ]:
subax1 = plt.subplot(111)
pos = nx.circular_layout(grLO)

edges = grLO.edges()
colors = [grLO[u][v]['art'] for u,v in edges]
for i in range(len(colors)):
  
    if colors[i] == 'neestedComp':
        colors[i] = 'r'
  
    if colors[i] == 'required_comp':
        colors[i] = 'g'
  
    if colors[i] == 'required_ueber_comp':
        colors[i] = 'b'
  
    if colors[i] == 'neestedueberComp':
        colors[i] = 'r'
  
    if colors[i] == 'offered_comp':
        colors[i] = 'g'
  
    if colors[i] == 'offered_ueber_comp':
        colors[i] = 'g'
  
# print list
print(colors)



nx.draw(grLO,pos=pos,  edge_color=colors, with_labels=True)

In [ ]:
#'9ueber', '6ueber', '4ueber', '11', '10Lo', '9', '8Lo', '20', '7Lo', '6', '5Lo', '18', '4Lo', '2ueber', '2Lo', '1ueber', '1'
nx.simple_cycles(grLO)

In [ ]:
nx.recursive_simple_cycles(grLO)

In [ ]:
find_something_by_id(my_LO_complete, '2')


In [ ]:
find_something_by_id(my_competencies.competencies, '6')



In [ ]:
find_something_by_id(my_competencies.ueber_competencies, '3')

In [ ]:
nx.all_pairs_shortest_path(grLO)

In [ ]:
sp = dict(nx.all_pairs_shortest_path(grLO))



In [ ]:
H = nx.DiGraph(((u, v, e) for u,v,e in grLO.edges(data=True) if e['art'] == 'offered_ueber_comp'or e['art'] == 'offered_comp' or e['art'] == 'required_ueber_comp'or e['art'] == 'required_comp'))

nx.draw(H,with_labels=True)


In [ ]:
sorted(list(nx.descendants(H, 'Know nothing')))


In [ ]:
G_reversed = nx.reverse(grLO)

In [ ]:
paths = nx.all_simple_paths(grLO, source='Know nothing', target='9ueber', cutoff=10)

In [ ]:
print (paths)
print(list(paths))

In [ ]:
mylist=list(nx.bfs_tree(grLO, source='Know nothing', depth_limit=20).edges())
print(mylist)
G=nx.from_edgelist(list(nx.bfs_tree(grLO, source='Know nothing', depth_limit=20).edges()))
G1 = nx.DiGraph()
G1.add_edges_from(G.edges)
pos = nx.planar_layout(G1)
nx.draw(G1, with_labels=True, pos = pos)

In [ ]:
find_something_by_id(my_LO_complete, '1')

In [ ]:
lst=[]
for x,y in mylist: 
    if x.find('Lo') != -1:
        
        lst.append(x) if x not in lst else lst
print(lst)    

In [ ]:
find_something_by_id(my_LO_complete, '6')

In [ ]:
find_something_by_id(my_LO_complete, '17')

In [ ]:
find_something_by_id(my_competencies.competencies, '27')

In [ ]:


from collections import defaultdict
from operator import itemgetter


B = nx.dag_to_branching(G1)
sources = defaultdict(set)
for v, source in B.nodes(data="source"):
    sources[source].add(v)
print (sources)



In [ ]:
nx.draw(B, with_labels=True)



In [ ]:
nx.draw(B, with_labels=True)